In [ ]:
data = "../4_basic_image_recognition/data/mnist.pkl"
train_data = MnistData(data, True, 'training')
test_data = MnistData(data, False, 'test')

## Dropout
<br>


In [ ]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28*28], name='x-intput')
    y = tf.placeholder(tf.int64, [None], name='y-input')
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='x-input-image')

keep_prob = tf.placeholder(tf.float32, name='dropout-rate')
    
# for tensorboard image visualization
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1]) # see x.shape

with tf.name_scope('layers'):
    # add a new convolutional layer
    conv_1 = tf.layers.conv2d(inputs=x_image, 
                              filters=32, 
                              kernel_size=(5, 5),
                              padding = 'same',
                              activation=None,
                              name= 'conv1')
    # add batch normalization
    conv_1 = tf.layers.batch_normalization(conv_1, training=True)
    # use activation function
    conv_1 = tf.nn.relu(conv_1)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv_1,
                                    pool_size=(2, 2),
                                    strides=(2,2),
                                    name='pool1')

    conv_2 = tf.layers.conv2d(inputs=pool1,
                              filters=64,
                              kernel_size=(5, 5),
                              padding = 'same',
                              activation=None,
                              name='conv2')
    
    conv_2 = tf.layers.batch_normalization(conv_2, training=True)
    conv_2 = tf.nn.relu(conv_2)

    pool2 = tf.layers.max_pooling2d(inputs=conv_2,
                                    pool_size=(2,2),
                                    strides=(2,2),
                                    name= 'pool2')

    # fc layer1
    flatten = tf.layers.flatten(pool2, name= 'fc1_flatten')
    
    # dropout bunch of neurons
    flatten_dropout = tf.layers.dropout(flatten, rate=keep_prob, training=True)
    
    # fc layer2
    y_ = tf.layers.dense(flatten_dropout, 10, name= 'fc2_flatten')

with tf.name_scope('loss'):
    #sparse_softmax_cross_entropy include one_hot encoding
    loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

with tf.name_scope('accuracy'):
    predict = tf.argmax(y_, 1)
    correct_prediction = tf.equal(predict, y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    with tf.name_scope('accuracy'):
        tf.summary.scalar('accuracy', accuracy)
    
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)


In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 50
n_batch = train_data._num_examples // batch_size
data = "../4_basic_image_recognition/data/mnist.pkl"
x_train, y_train, x_test, y_test = load(data)

# train 10k: %
with tf.Session() as sess:
    sess.run(init)
    # start training
    for epoch in range(10):
        for batch in range(n_batch):
            batch_data, batch_labels = train_data.next_batch(batch_size)
            sess.run([train_op],feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.7})
        
        ########## need more than 16GB memory ##########
        #test_acc = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
        #train_acc = sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(test_acc) + ",Training Accuracy " + str(train_acc))    
    